In [1]:
from dnn_structure import *

In [2]:
import numpy as np
import sys
sys.path.append("../src")
from Load_data.xml_utils import parse_cvat_xml_all_labels, label_Y_binary
from Load_data.data_loader import load_and_resize_images, build_label_array


In [3]:
TRAIN_XML = "../EIDSeg_Dataset/data/train/train.xml"
TEST_XML = "../EIDSeg_Dataset/data/test/test.xml"
TRAIN_IMAGES = "../EIDSeg_Dataset/data/train/images/default"
TEST_IMAGES = "../EIDSeg_Dataset/data/test/images/default"

IMAGE_SIZE = (64, 64)
NUM_ITER = 2000

## Load and parse XML labels

In [4]:
labels_train_raw = parse_cvat_xml_all_labels(TRAIN_XML)
labels_test_raw = parse_cvat_xml_all_labels(TEST_XML)

Y_train_map = label_Y_binary(labels_train_raw)
Y_test_map = label_Y_binary(labels_test_raw)

## Load and preprocess images

In [5]:
X_train_org, ordered_train = load_and_resize_images(TRAIN_IMAGES, size=IMAGE_SIZE)
X_test_org, ordered_test = load_and_resize_images(TEST_IMAGES, size=IMAGE_SIZE)

Y_train_org = build_label_array(ordered_train, Y_train_map)
Y_test_org = build_label_array(ordered_test, Y_test_map)

Final X shape: (2612, 64, 64, 3)
Final X shape: (327, 64, 64, 3)


## Flatten Inputs

In [6]:
train_x = X_train_org.reshape(X_train_org.shape[0], -1).T
test_x = X_test_org.reshape(X_test_org.shape[0], -1).T

print(train_x.shape, Y_train_org.shape)
print(test_x.shape, Y_test_org.shape)

(12288, 2612) (1, 2612)
(12288, 327) (1, 327)


## L-layer Neural Network


In [7]:
#layers_dims = [12288, 20, 7, 5, 1] 
layers_dims = [12288, 64, , 5, 1] 

In [10]:

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (n_x, number of examples)
    Y -- true "label" vector (containing 1 if cat, 0 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization.

    parameters = initialize_parameters_deep(layers_dims)
        
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_model_forward(X, parameters)
                
        # Compute cost.
        cost = compute_cost(AL,Y)
            
        # Backward propagation.
        grads = L_model_backward(AL, Y, caches)
        
        # Update parameters.

        parameters = update_parameters(parameters, grads, learning_rate)
                        
        # Print the cost every 100 iterations and for the last iteration
        if print_cost and (i % 100 == 0 or i == num_iterations - 1):
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0:
            costs.append(cost)
    
    return parameters, costs

In [12]:
parameters, costs = L_layer_model(train_x, Y_train_org, layers_dims, num_iterations = 3000, print_cost = True)

Cost after iteration 0: 0.7241635561801133
Cost after iteration 100: 0.6661479180759993
Cost after iteration 200: 0.6474481262272356
Cost after iteration 300: 0.6345250663622681
Cost after iteration 400: 0.6254689330587386
Cost after iteration 500: 0.6189893996591092
Cost after iteration 600: 0.6142055357837776
Cost after iteration 700: 0.6103269485193432
Cost after iteration 800: 0.6070572789401693
Cost after iteration 900: 0.6036781399784718
Cost after iteration 1000: 0.5995746399237991
Cost after iteration 1100: 0.5951724741465234
Cost after iteration 1200: 0.5896547407833196
Cost after iteration 1300: 0.5834768820575619
Cost after iteration 1400: 0.5766653510185422
Cost after iteration 1500: 0.5698711553518849
Cost after iteration 1600: 0.5629031238036939
Cost after iteration 1700: 0.555555056615481
Cost after iteration 1800: 0.5478432439701002
Cost after iteration 1900: 0.5401285694072493
Cost after iteration 2000: 0.5340149264824757
Cost after iteration 2100: 0.5303676118614118
C

In [13]:
pred_train = predict(train_x, Y_train_org, parameters)

Accuracy: 0.7289433384379786


In [14]:
pred_test = predict(test_x, Y_test_org, parameters)

Accuracy: 0.7370030581039757


In [15]:
layers_dims = [12288, 32, 8, 1]
parameters, costs = L_layer_model(train_x, Y_train_org, layers_dims,learning_rate = 0.005, num_iterations = 3000, print_cost = True)

Cost after iteration 0: 0.6997482480275381
Cost after iteration 100: 0.6741384924699683
Cost after iteration 200: 0.6094007272448337
Cost after iteration 300: 0.6032148849916655
Cost after iteration 400: 0.5979940844763588
Cost after iteration 500: 0.593288757498294
Cost after iteration 600: 0.5887572053603997
Cost after iteration 700: 0.5843336164678468
Cost after iteration 800: 0.5799381838670103
Cost after iteration 900: 0.5749674904472419
Cost after iteration 1000: 0.5698698579101686
Cost after iteration 1100: 0.5645610239021701
Cost after iteration 1200: 0.5588385486763775
Cost after iteration 1300: 0.552669995753617
Cost after iteration 1400: 0.5460135740749259
Cost after iteration 1500: 0.5387776210909023
Cost after iteration 1600: 0.5309011274380034
Cost after iteration 1700: 0.5223348511308217
Cost after iteration 1800: 0.5130265777568344
Cost after iteration 1900: 0.5028467317214917
Cost after iteration 2000: 0.49170815284250297
Cost after iteration 2100: 0.4797982640997426
C

In [16]:
pred_train = predict(train_x, Y_train_org, parameters)

Accuracy: 0.7415773353751915


In [17]:
pred_test = predict(test_x, Y_test_org, parameters)

Accuracy: 0.7278287461773701


In [18]:
layers_dims = [12288, 20, 7, 5, 1] 
parameters, costs = L_layer_model(train_x, Y_train_org, layers_dims,learning_rate = 0.008, num_iterations = 3500, print_cost = True)

Cost after iteration 0: 0.636104091418676
Cost after iteration 100: 0.5817297824912824
Cost after iteration 200: 0.5682603793620732
Cost after iteration 300: 0.5584356704737771
Cost after iteration 400: 0.5499503509074896
Cost after iteration 500: 0.5419398292623027
Cost after iteration 600: 0.5289647715143653
Cost after iteration 700: 0.5191224412075178
Cost after iteration 800: 0.5014854937019197
Cost after iteration 900: 0.5078403450081426
Cost after iteration 1000: 0.4977298868604474
Cost after iteration 1100: 0.48494802281877375
Cost after iteration 1200: 0.4826871703080125
Cost after iteration 1300: 0.42735344682676
Cost after iteration 1400: 0.4778233456725642
Cost after iteration 1500: 0.4341173208781856
Cost after iteration 1600: 0.44458053939084896
Cost after iteration 1700: 0.439247606573252
Cost after iteration 1800: 0.5873562237522952
Cost after iteration 1900: 0.4148386360778437
Cost after iteration 2000: 0.4786291706183567
Cost after iteration 2100: 0.5128041824956233
Co

In [19]:
pred_train = predict(train_x, Y_train_org, parameters)

Accuracy: 0.9345329249617154


In [20]:
pred_test = predict(test_x, Y_test_org, parameters)

Accuracy: 0.7339449541284404


In [1]:
print('Hello')

Hello
